In [5]:
!pip install fastapi nest-asyncio uvicorn

In [4]:
!pip install -U kaleido

In [3]:
!pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.4 MB/s eta 0:00:00


In [6]:
!pip install -q pyngrok

In [7]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

In [8]:
import pandas as pd

In [9]:
# Como estamos trabajando en colab y tengo los archivos en el drive importamos la libreria y montamos
from google.colab import drive #para utilizar google drive como fuente de archivos
drive.mount('/content/drive') #montamos la carpeta del drive

Mounted at /content/drive


In [10]:
games = pd.read_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/gamesv5.parquet')
reviews = pd.read_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/reviews.parquet')
items = pd.read_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/itemsv2.parquet')

In [11]:
generos = list(games.drop(columns=['item_id','release_date']).columns)

In [12]:
ids =pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/idsnv2.csv')

In [13]:
iid=list(set(items['item_id']))

In [14]:
lanio=list(set(reviews['posted'].to_list()))

In [15]:
with open('/content/drive/MyDrive/Henry/ntk.txt', 'r') as archivo:
    token = archivo.read()

In [16]:
@app.get('/')
async def root():
    return {'hello': 'world'}

In [17]:
@app.get('/PlayTimeGenre/{genero}')
async def PlayTimeGenre(genero:str):
  genero=genero.upper()
  if genero in generos:
    lgames = games.loc[games[genero] == 1, ['release_date', 'item_id']]
    lhoras = items.groupby('item_id')['playtime_forever'].sum().reset_index()
    dfplay = pd.merge(lgames, lhoras, on='item_id', how='inner')
    df_horas = dfplay.groupby('release_date')['playtime_forever'].sum().reset_index()
    df_horas_ordenado = df_horas.sort_values(by='playtime_forever', ascending=False)
    resultado=df_horas_ordenado.head(1)
    prin = "Año de lanzamiento con más horas jugadas para Género " + genero
    anio = int(resultado['release_date'])
    return {prin : anio}
  else:
    return {"El Genero indicado no se encuentra en la base de datos":genero}

In [18]:
@app.get('/UserForGenre/{genero}')
async def UserForGenre(genero:str):
  genero=genero.upper()
  if genero in generos:
    dfgames = games[['item_id','release_date',genero]]
    dfgames = dfgames[games[genero]==1]
    dfuserg = pd.merge(dfgames,items,on='item_id',how='inner')
    uhoras = dfuserg.groupby('user_id')['playtime_forever'].sum().reset_index()
    uhoras = uhoras.sort_values(by='playtime_forever', ascending=False)
    uhoras = uhoras.head(1)
    uhoras = uhoras['user_id'].iloc[0]
    dfuserg = dfuserg[dfuserg['user_id']==uhoras]
    dfuserg = dfuserg.groupby('release_date')['playtime_forever'].sum().reset_index()
    dfuserg = dfuserg.sort_values(by='release_date', ascending=False)
    dfuserg = dfuserg[dfuserg['playtime_forever']>0]
    dfuserg = dfuserg.astype(int)
    dfuserg.rename(columns={'release_date': 'Año', 'playtime_forever':'Horas'}, inplace=True)
    di = dfuserg.apply(lambda row: row.to_dict(), axis=1).tolist()
    prin="Usuario con más horas jugadas para Género " + genero
    return {prin:uhoras,"Horas Jugadas":di}
  else:
    return {"El Genero indicado no se encuentra en la base de datos ":genero}

In [19]:
@app.get('/UsersRecommend/{anio}')
async def UsersRecommend(anio:int):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == True) & (reviews['sentiment_analysis'] >0)].copy()
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
      if g in iid:
        top_games[i]=items[items['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
      else:
        top_games[i]=ids[ids['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [20]:
@app.get('/UsersNotRecommend/{anio}')
async def UsersNotRecommend(anio:int):
  if anio in lanio:
    dfa = reviews[(reviews['posted'] == anio) & (reviews['recommend'] == False) & (reviews['sentiment_analysis'] ==0)].copy()
    dfa['sentiment_analysis']=1
    dfa = dfa.groupby('item_id')['sentiment_analysis'].sum().reset_index()
    dfa = dfa.sort_values('sentiment_analysis', ascending=False)
    top_games = dfa.head(3)['item_id'].to_list()
    for i,g in enumerate(top_games):
      if g in iid:
        top_games[i]=items[items['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
      else:
        top_games[i]=ids[ids['item_id'] == top_games[i]].head(1)['item_name'].iloc[0]
    if len(top_games)<2:
      return{'Puesto 1':top_games[0],'Puesto 2':'','Puesto 3':''}
    elif len(top_games)<3:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':''}
    else:
      return{'Puesto 1':top_games[0],'Puesto 2':top_games[1],'Puesto 3':top_games[2]}
  else:
    return {f'El año indicado no se encuentra en la base de datos: ':anio}

In [21]:
@app.get('/sentiment_analysis/{anio}')
async def sentiment_analysis(anio:int):
  dfs = pd.merge(reviews[['item_id','sentiment_analysis']],games[['item_id','release_date']].drop_duplicates(),on='item_id',how='inner')
  dfs = dfs[dfs['release_date']==anio]
  dfs = dfs.groupby('sentiment_analysis').size().reset_index(name='count')
  dfs = dfs.sort_values('sentiment_analysis', ascending=True)
  dfs = dfs['count'].to_list()
  return{'Negative':dfs[0],'Neutral':dfs[1],'Positive':dfs[2]}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.set_auth_token(token)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [637]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://adfa-104-155-193-173.ngrok-free.app
INFO:     2800:c70:88:ef4f:951a:9ae2:25dd:4a23:0 - "GET / HTTP/1.1" 200 OK
INFO:     2800:c70:88:ef4f:951a:9ae2:25dd:4a23:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2800:c70:88:ef4f:951a:9ae2:25dd:4a23:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2800:c70:88:ef4f:951a:9ae2:25dd:4a23:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2800:c70:88:ef4f:951a:9ae2:25dd:4a23:0 - "GET /PlayTimeGenre/Rpg HTTP/1.1" 200 OK
INFO:     2800:c70:88:ef4f:951a:9ae2:25dd:4a23:0 - "GET /PlayTimeGenre/AnImE HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [637]
